## Rubric

| Criteria                       | Ratings                                                                                                                                                                                   | Pts |
| ------------------------------ | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --- |
| 1a BST Max                     | 3 pts Full Marks<br>0 pts No Marks                                                                                                                                                        | 3   |
| 1b BST Min                     | 3 pts Full Marks<br>0 pts No Marks                                                                                                                                                        | 3   |
| 1c BST Sum                     | 7 pts Full Marks<br>0 pts No Marks                                                                                                                                                        | 7   |
| 1d BST Height                  | 7 pts Full Marks<br>0 pts No Marks                                                                                                                                                        | 7   |
| BST Validator                  | 20 pts Full Marks<br>15 pts does not work for all cases<br>0 pts No Marks                                                                                                                 | 20  |
| Character frequency heap sort  | 20 pts Full Marks<br>10 pts Character order<br>characters must be in ascending order<br>0 pts No Marks                                                                                    | 20  |
| Median Tracker                 | 20 pts Full Marks<br>15 pts Efficiency<br>Must have insert O(log n) efficiency and median O(1) efficiency<br>10 pts Methods<br>Must implement insert and median methods<br>0 pts No Marks | 20  |
| Find the longest common prefix | 20 pts Full Marks<br>15 pts could be more efficient<br>0 pts No Marks                                                                                                                     | 20  |

**Total Points: 100**

## 2. Binary Search Tree Validator

Write a function that accepts a binary tree and verifies whether it fulfills binary search tree conditions.

## 3. Heap-based Frequency Sort

Write a function that accepts a string and returns an array of the characters in the string sorted by frequency (from most frequent to least frequent).

In case of a tie, characters should be ordered in ascending alphabetical order
You must use a heap to sort the characters

You may use the heap in the dsa package, Python's heapq package or write your own. 

**Example**: 
Input:
```txt
"open sesame"
```

Output:
```txt
['e', 's', ' ', 'a', 'm', 'n', 'o', 'p']
```

## 4.  Heap-based Median Tracker

Write functions/methods that efficiently computes the median of a list of numbers using a heap-based approach.

- The median is the middle value in a sorted list
- If there are an odd number of elements, return the middle value
- If there are an even number of elements, return the average of the two middle values
- While sorting the list is acceptable for static datasets, it becomes inefficient for dynamic or real-time updates. Your solution should be designed to handle such updates efficiently using heaps.

**Requirements**:

- Implement an efficient strategy for maintaining the median as new numbers are added
- One function should support inserting numbers one at a time
- Another function should return the current median

**Example**:

```python
tracker = MedianTracker()
numbers = [30, 20, 10, 50, 40]

for num in numbers:
    tracker.insert(num)
    print(f"Inserted {num}, current median: {tracker.get_median()}")
```

**Output**
```txt
Inserted 30, current median: 30
Inserted 20, current median: 25.0
Inserted 10, current median: 20
Inserted 50, current median: 25.0
Inserted 40, current median: 30
```

## 5. Longest Common Prefix in a Trie

Write a function that accepts an array of words, stores them in a trie and returns the longest common prefix. Write it so that it performs efficiently.

For example, given the array

```python
words = ["apple", "appetite", "apparatus", "appliance"]
```

The function should return

```sh
"app"
```



## Custom Imports

In [34]:
from __future__ import annotations

from typing import Any, Callable, Generic, List, Optional, Protocol, TypeVar, cast

T = TypeVar("T")


# A protocol expressing that a type supports math comparisons
class SupportsMath(Protocol):
    # Arithmetic operations
    def __add__(self: T, other: T) -> T: ...
    def __sub__(self: T, other: T) -> T: ...
    def __mul__(self: T, other: T) -> T: ...
    def __truediv__(self: T, other: T) -> T: ...
    def __floordiv__(self: T, other: T) -> T: ...
    def __mod__(self: T, other: T) -> T: ...
    def __pow__(self: T, other: T, modulo: Any = ...) -> T: ...

    # Unary operations
    def __neg__(self: T) -> T: ...
    def __pos__(self: T) -> T: ...
    def __abs__(self: T) -> T: ...

    # Comparisons
    def __lt__(self, other: Any) -> bool: ...
    def __le__(self, other: Any) -> bool: ...
    def __gt__(self, other: Any) -> bool: ...
    def __ge__(self, other: Any) -> bool: ...
    def __eq__(self, other: Any) -> bool: ...
    def __ne__(self, other: Any) -> bool: ...


M = TypeVar("M", bound=SupportsMath)


# -----------------------------
# Node Base Classes
# -----------------------------
class NodeBase(Generic[T]):
    """Base class for nodes in linked lists."""

    def __init__(self, value: T):
        self.value: T = value


# -----------------------------
# Binary Tree Node
# -----------------------------
class BinaryTreeNode(NodeBase[M]):
    """A node in a binary tree, inheriting from NodeBase."""

    __slots__ = ("value", "left", "right")

    def __init__(
        self,
        value: M,
        left: Optional[BinaryTreeNode[M]] = None,
        right: Optional[BinaryTreeNode[M]] = None,
    ) -> None:
        super().__init__(value)
        self.left = left
        self.right = right

    def __repr__(self) -> str:
        return f"BinaryTreeNode({self.value!r})"


class BinarySearchTree(Generic[M]):
    """Binary search tree wrapper class."""

    __slots__ = ("_root", "_size")

    def __init__(self, root: Optional[BinaryTreeNode[M]] = None) -> None:
        self._root = root
        self._size = 0 if root is None else self._compute_size(root)

    @property
    def root(self) -> Optional[BinaryTreeNode[M]]:
        return self._root

    @property
    def size(self) -> int:
        return self._size

    def is_empty(self) -> bool:
        return self._root is None

    def _compute_size(self, node: Optional[BinaryTreeNode[M]]) -> int:
        if node is None:
            return 0
        return 1 + self._compute_size(node.left) + self._compute_size(node.right)

    # -------------------------------
    # Insert (simple binary search tree insert)
    # -------------------------------
    def insert(self, value: M) -> None:
        """Insert a value into the binary tree (BST style)."""

        def _insert(node: Optional[BinaryTreeNode[M]], value: M) -> BinaryTreeNode[M]:
            if node is None:
                return BinaryTreeNode(value)
            if value < node.value:
                node.left = _insert(node.left, value)
            else:
                node.right = _insert(node.right, value)
            return node

        self._root = _insert(self._root, value)
        self._size += 1

    # -------------------------------
    # Traversals
    # -------------------------------
    def inorder(self, visit: Callable[[M], None]) -> None:
        def _inorder(node: Optional[BinaryTreeNode[M]]):
            if node:
                _inorder(node.left)
                visit(node.value)
                _inorder(node.right)

        _inorder(self._root)

    def preorder(self, visit: Callable[[M], None]) -> None:
        def _preorder(node: Optional[BinaryTreeNode[M]]):
            if node:
                visit(node.value)
                _preorder(node.left)
                _preorder(node.right)

        _preorder(self._root)

    def postorder(self, visit: Callable[[M], None]) -> None:
        def _postorder(node: Optional[BinaryTreeNode[M]]):
            if node:
                _postorder(node.left)
                _postorder(node.right)
                visit(node.value)

        _postorder(self._root)

    # -------------------------------
    # Search
    # -------------------------------
    def find_max(self) -> Optional[BinaryTreeNode[M]]:
        """
        Finds the maximum value in the binary search tree
        Time Complexity: Avg. O(logn) since DFS is used to traverse each level to the max value
                         O(n) worst case for an unbalanced BST
        Space Complexity: O(1) since no extra space is required
        """

        # Max value will be found by traversing to the rightmost node
        def _find_max(node: Optional[BinaryTreeNode[M]]) -> Optional[BinaryTreeNode[M]]:
            if node is None:
                return None

            if node.right is None:
                return node

            return _find_max(node.right)

        return _find_max(self._root)

    def find_min(self) -> Optional[BinaryTreeNode[M]]:
        """
        Finds the minimum value in the binary search tree
        Time Complexity: Avg. O(logn) since DFS is used to traverse each level to the max value
                         O(n) worst case for an unbalanced BST
        Space Complexity: O(1) since no extra space is required
        """

        # Max value will be found by traversing to the leftmost node
        def _find_min(node: Optional[BinaryTreeNode[M]]) -> Optional[BinaryTreeNode[M]]:
            if node is None:
                return None

            if node.left is None:
                return node

            return _find_min(node.left)

        return _find_min(self._root)

    def sum(self) -> M:
        """
        Finds the sum of all values binary search tree
        Time Complexity: O(n) since all nodes must be traversed
        Space Complexity: O(1) since no extra space is required
        """
        sum = cast(M, 0)

        def _add(val: M) -> None:
            nonlocal sum
            sum += val

        self.preorder(_add)
        return sum

    def height(self) -> int:
        """
        Returns the maximum height of the binary tree from an input node
        Time Complexity: O(n) since all nodes must be traversed to find the largest height (postorder traversal)
        Space Complexity: O(1) since no extra space is required
        """
        max_height = 0

        def _preorder(node: Optional[BinaryTreeNode[M]], current_height: int):
            nonlocal max_height

            # Preorder traverse until we reach a leaf
            if node:
                current_height += 1  # increment the height
                _preorder(node.left, current_height)
                _preorder(node.right, current_height)

            # At the leaf, check the heights
            # and override the max height if applicable
            if current_height > max_height:
                max_height = current_height

        _preorder(self._root, 0)
        return max_height

    # -------------------------------
    # Utility
    # -------------------------------
    def to_list(self) -> List[M]:
        """Return all values of the tree in inorder as a list."""
        values: List[M] = []
        self.inorder(values.append)
        return values

    def __repr__(self) -> str:
        return f"BinaryTree({self.to_list()})"

    def pretty_print(self) -> str:
        if not self._root:
            return "<empty>"

        def _display(
            node: Optional["BinaryTreeNode[M]"],
        ) -> tuple[list[str], int, int, int]:
            """
            Returns:
                lines: list of strings for this subtree
                width: total width
                height: total height
                middle: horizontal position of root
            """
            if node is None:
                return [], 0, 0, 0

            line = f"┌{node.value}┐"
            width = len(line)

            if node.left is None and node.right is None:
                return [line], width, 1, width // 2

            # Recursively display left and right
            left_lines, left_width, left_height, left_middle = _display(node.left)
            right_lines, right_width, right_height, right_middle = _display(node.right)

            # Compute new width and height
            height = max(left_height, right_height) + 2
            middle = left_width + width // 2 + 1 if node.left else width // 2

            # Pad left/right lines
            left_lines += [" " * left_width] * (right_height - left_height)
            right_lines += [" " * right_width] * (left_height - right_height)

            # Combine lines
            lines = [" " * left_width + line + " " * right_width]  # root line
            for l, r in zip(left_lines, right_lines):
                lines.append(l + " " * width + r)
            return lines, left_width + width + right_width, height, middle

        lines, _, _, _ = _display(self._root)
        return "\n".join(lines)


## 1. Binary Search Tree Functions

Write the following binary search tree functions to:

- Return the minimum value
- Return the maximum value
- Return the sum of all values
- Return the height (The height of a BST is the number of edges on the longest path from the root node to a leaf node)

---
### Answers

In [45]:
import random

In [51]:
bst = BinarySearchTree()
sample = random.sample(range(1,50), 20)
print(f"Sample: {sample}")

for s in sample:
   bst.insert(s)

print(bst.pretty_print())

print(f"Min: {bst.find_min()}")
print(f"Max: {bst.find_max()}")
print(f"Height: {bst.height()}")
print(f"Sum: {bst.sum()}")

Sample: [44, 12, 7, 23, 13, 39, 26, 33, 30, 36, 5, 38, 25, 29, 28, 42, 2, 37, 9, 41]
                                                                        ┌44┐
            ┌12┐                                                            
      ┌7┐           ┌23┐                                                    
   ┌5┐   ┌9┐    ┌13┐                                        ┌39┐            
┌2┐                         ┌26┐                                    ┌42┐    
                        ┌25┐                ┌33┐                ┌41┐        
                                        ┌30┐    ┌36┐                        
                                    ┌29┐                ┌38┐                
                                ┌28┐                ┌37┐                    
Min: BinaryTreeNode(2)
Max: BinaryTreeNode(44)
Height: 9
Sum: 519
